## Priprema podataka za obučavanje modela

U ovom dijelu vrši se priprema podataka za obuku modela. Sljedeći koraci su izvršeni:

1. **Konverzija kategoričkih varijabli u numeričke (definicija)**: Definisana je funkcija `convert_dataframe` koja transformiše kategoričke varijable u numeričke. Također, funkcija sprema .txt datoteku koja sadrži mapiranje starih kategoričkih varijabli na nove numeričke vrijednosti, što se može koristiti za inferenciju kasnije. Funkcija prolazi kroz sve kolone u DataFrame-u i provjerava tip podataka u koloni. Ako je tip podataka 'object', kolona se mapira na brojeve pomoću kategoričkih oznaka i pohranjuje se mapiranje za tu kolonu. Ako je tip podataka numerički ili boolean, kolona se konvertuje u odgovarajući tip podataka. Na kraju se novo konvertovani DataFrame sačuva kao CSV datoteka, a mapiranje kolona se sprema u tekstualnu datoteku.

2. **Importovanje skupa podataka**: Učitavamo CSV datoteku sa podacima (`part_geocoded.csv`) u DataFrame.

3. **Uklanjanje podataka o stanovima bez lokacijskih informacija**: Uklanjamo podatke o stanovima kod kojih nismo mogli pronaći informacije o lokaciji, tj. podatke koji imaju nulte vrijednosti za koordinate.

4. **Uklanjanje kolona sa NaN vrijednostima i kolone sa koordinatama**: Uklanjamo kolone koje sadrže NaN vrijednosti i koordinate, jer više nisu korisne za daljnju obradu.

5. **Konverzija kategoričkih varijabli u numeričke**: Pozivamo funkciju `convert_dataframe` sa DataFrame-om kao ulaznim parametrom. Funkcija konvertuje preostale kategoričke varijable u numeričke, koristeći one-hot encoding za kategoričke varijable. Konvertovani DataFrame se spašava kao CSV datoteka, a mapiranje kolona se spašava kao tekstualna datoteka.

6. **Podjela skupa podataka na trening i test skup**: Učitavamo potrebne biblioteke za podjelu skupa podataka na trening i test skup. Nakon toga, vršimo podjelu skupa podataka na trening i validacijski skup pomoću funkcije `train_test_split`. Ulazni skup podataka se dijeli na ulazne i ciljne varijable (X i Y), te se dobijeni skupovi dijele na trening i validacijski skup.

Ovaj postupak pripreme podataka omogućava nam da konvertujemo kategoričke varijable u numeričke, uklonimo nepotrebne kolone i podijelimo skup podataka na trening i test skup, što je neophodno za daljnju obuku modela i evaluaciju performansi.


In [7]:
import pandas as pd
import numpy as np

In [177]:
def convert_dataframe(df, csv_filename, mapping_filename):
    """
    This function transforms categorical variables to numeric. It also saves .txt file that contains mapping 
    of old categorical variables to new numeric, so this can be used for inference later.
    """
    # Create a new DataFrame to store the converted columns
    converted_df = pd.DataFrame()

    # Create a dictionary to store the column mappings
    column_mappings = {}

    # Iterate over the columns of the input DataFrame
    for column in df.columns:
        if df[column].dtype == 'object':  # Check if the column contains categorical values
            # Map categorical values to numbers
            mapped_column = df[column].astype('category').cat.codes.astype('float')

            # Store the mapping for the column
            column_mappings[column] = dict(enumerate(df[column].astype('category').cat.categories))

        elif np.issubdtype(df[column].dtype, np.number):  # Check if the column contains numeric values
            # Convert the column to float
            mapped_column = df[column].astype('float')

        elif df[column].dtype == 'bool':  # Check if the column contains boolean values
            # Convert boolean values to 0 or 1
            mapped_column = df[column].astype('int')

        else:
            # Skip columns that are neither categorical, numeric, nor boolean
            print(column)
            continue

        # Add the converted column to the new DataFrame
        converted_df[column] = mapped_column

    # Save the converted DataFrame to CSV
    converted_df.to_csv(csv_filename, index=False)

    # Save the column mappings to a text file
    with open(mapping_filename, 'w') as file:
        for column, mapping in column_mappings.items():
            if df[column].dtype != 'bool':  # Skip boolean columns in the mapping
                file.write(f'{column}:\n')
                for key, value in mapping.items():
                    file.write(f'{key}: {value}\n')
                file.write('\n')

    print("Conversion completed. DataFrame and mappings saved successfully.")

In [178]:
# Importing dataset
df = pd.read_csv('part_geocoded.csv')

# Removing appatrment data for apartments we couldn't find location info
df = df[df.object_longitude!=0]


# Removing columns with NaN values and coordinates columns since they are no longer useful
df = df.drop(['Unnamed: 0', 'stanje', 'godina_izgradnje', 'adresa', 'vrsta_poda', 'object_latitude', 'object_longitude'], axis=1)
df = df.dropna()

In [179]:
df

,cijena,kvadratura,broj_soba,sprat,namjesten,vrsta_grijanja,kvadratura_balkona,blindirana_vrata,internet,kablovska_tv,...,novogradnja,tel_prikljucak,uknjizeno_zk,plin,podrum_tavan,video_nadzor,ostava_spajz,alarm,nedavno_adaptiran,udaljenost_od_centra
0,295000.0,70.0,4.0,5,Namješten,Plin,16.0,False,True,True,...,False,False,True,True,True,False,False,False,True,3813.9
1,489762.0,116.0,4.0,6,Namješten,Centralno (gradsko),6.0,True,True,True,...,True,True,True,False,True,False,True,False,False,6415.6
3,227700.0,69.0,3.0,1,Namješten,Plin,9.0,True,True,True,...,False,True,True,True,False,False,False,False,False,9655.4
4,610000.0,134.0,3.0,2,Namješten,Centralno (gradsko),0.0,True,True,True,...,True,True,True,True,True,True,True,True,True,9140.5
14,397000.0,85.0,3.0,1,Namješten,Centralno (gradsko),8.0,True,True,True,...,False,True,True,False,False,False,True,False,True,4315.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,190000.0,58.0,2.0,Suteren,Polunamješten,Centralno (Plin),0.0,True,True,True,...,False,True,True,True,False,False,True,False,True,1378.9
1531,137800.0,53.0,2.0,1,Polunamješten,Struja,6.0,True,True,True,...,True,True,True,False,True,True,True,True,True,13134.7
1533,330000.0,76.0,3.0,8,Polunamješten,Centralno (gradsko),24.5,True,True,True,...,False,True,True,False,False,False,False,False,False,4315.0
1534,160000.0,48.0,2.0,2,Polunamješten,Plin,4.0,True,True,True,...,True,True,True,True,False,True,False,False,False,10594.8


In [180]:
# Convert categorical variables to numeric
convert_dataframe(df, 'converted_dataframe.csv', 'column_mappings.txt')

Conversion completed. DataFrame and mappings saved successfully.


In [181]:
# Read dataset
df_final = pd.read_csv('converted_dataframe.csv')
df_final.shape

(1049, 22)

In [182]:
df_final

,cijena,kvadratura,broj_soba,sprat,namjesten,vrsta_grijanja,kvadratura_balkona,blindirana_vrata,internet,kablovska_tv,...,novogradnja,tel_prikljucak,uknjizeno_zk,plin,podrum_tavan,video_nadzor,ostava_spajz,alarm,nedavno_adaptiran,udaljenost_od_centra
0,295000.0,70.0,4.0,16.0,0.0,5.0,16.0,0,1,1,...,0,0,1,1,1,0,0,0,1,3813.9
1,489762.0,116.0,4.0,17.0,0.0,2.0,6.0,1,1,1,...,1,1,1,0,1,0,1,0,0,6415.6
2,227700.0,69.0,3.0,1.0,0.0,5.0,9.0,1,1,1,...,0,1,1,1,0,0,0,0,0,9655.4
3,610000.0,134.0,3.0,12.0,0.0,2.0,0.0,1,1,1,...,1,1,1,1,1,1,1,1,1,9140.5
4,397000.0,85.0,3.0,1.0,0.0,2.0,8.0,1,1,1,...,0,1,1,0,0,0,1,0,1,4315.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,190000.0,58.0,2.0,22.0,2.0,1.0,0.0,1,1,1,...,0,1,1,1,0,0,1,0,1,1378.9
1045,137800.0,53.0,2.0,1.0,2.0,6.0,6.0,1,1,1,...,1,1,1,0,1,1,1,1,1,13134.7
1046,330000.0,76.0,3.0,19.0,2.0,2.0,24.5,1,1,1,...,0,1,1,0,0,0,0,0,0,4315.0
1047,160000.0,48.0,2.0,12.0,2.0,5.0,4.0,1,1,1,...,1,1,1,1,0,1,0,0,0,10594.8


In [183]:
# Continuous variables are cijena, kvadratura i udaljenost_od_centra
# All other variables are categorical
# One hot encoding of categorical variables is performed here

from sklearn.preprocessing import OneHotEncoder


# Here select categorical variables
columns_to_cat=['broj_soba', 'blindirana_vrata','internet',
       'kablovska_tv', 'klima', 'lift', 'novogradnja', 'tel_prikljucak',
       'uknjizeno_zk', 'plin', 'podrum_tavan', 'video_nadzor', 'ostava_spajz',
       'alarm', 'nedavno_adaptiran']

df_final[columns_to_cat]=df_final[columns_to_cat].astype(str)

 
s = (df_final.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)
print('No. of. categorical features: ',
      len(object_cols))

OH_encoder = OneHotEncoder(sparse_output=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(df_final[object_cols]))
OH_cols.index = df_final.index
OH_cols.columns = OH_encoder.get_feature_names_out()
df_final = df_final.drop(object_cols, axis=1)
df_final = pd.concat([df_final, OH_cols], axis=1)

Categorical variables:
['broj_soba', 'blindirana_vrata', 'internet', 'kablovska_tv', 'klima', 'lift', 'novogradnja', 'tel_prikljucak', 'uknjizeno_zk', 'plin', 'podrum_tavan', 'video_nadzor', 'ostava_spajz', 'alarm', 'nedavno_adaptiran']
No. of. categorical features:  15


In [184]:
df_final

,cijena,kvadratura,sprat,namjesten,vrsta_grijanja,kvadratura_balkona,udaljenost_od_centra,broj_soba_0.0,broj_soba_1.0,broj_soba_1.5,...,podrum_tavan_0,podrum_tavan_1,video_nadzor_0,video_nadzor_1,ostava_spajz_0,ostava_spajz_1,alarm_0,alarm_1,nedavno_adaptiran_0,nedavno_adaptiran_1
0,295000.0,70.0,16.0,0.0,5.0,16.0,3813.9,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,489762.0,116.0,17.0,0.0,2.0,6.0,6415.6,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,227700.0,69.0,1.0,0.0,5.0,9.0,9655.4,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,610000.0,134.0,12.0,0.0,2.0,0.0,9140.5,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,397000.0,85.0,1.0,0.0,2.0,8.0,4315.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,190000.0,58.0,22.0,2.0,1.0,0.0,1378.9,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1045,137800.0,53.0,1.0,2.0,6.0,6.0,13134.7,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
1046,330000.0,76.0,19.0,2.0,2.0,24.5,4315.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1047,160000.0,48.0,12.0,2.0,5.0,4.0,10594.8,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [185]:
# Dividing dataset into training and testing
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
 
X = df_final.drop(['cijena'], axis=1)
Y = df_final['cijena']
 
# Split the training set into training and validation set
X_train, X_valid, Y_train, Y_valid = train_test_split(
    X, Y, train_size=0.8, test_size=0.2, random_state=0)

# Obuka modela i analiza rezultata

U ovom dijelu izvršava se obuka različitih modela i analiza njihovih rezultata. Izvršeni su sljedeći koraci:

1. **Prvi model - SVM (Support Vector Machine) Regresija**: Definisan je SVM regresijski model `model_SVR` i obučen na trening skupu koristeći funkciju `fit`. Nakon obuke, model je primijenjen na validacijski skup, a predviđanja su spremljena u varijablu `Y_pred`. Izračunata je srednja apsolutna postotna greška (mean absolute percentage error) između stvarnih i predviđenih vrijednosti koristeći funkciju `mean_absolute_percentage_error` i ona iznosi 0.3859895797701615.
- SVM regresija (Support Vector Machine Regresija) odabrana je  zbog sposobnosti modela da efikasno radi s podacima velikih dimenzija i da izgradi kompleksne granice odlučivanja. SVM se često koristi kada postoji linearna ili nelinearna zavisnost između ulaznih varijabli i ciljne varijable.

2. **Drugi model - Random Forest Regresija**: Definisana je Random Forest regresijski model `model_RFR` sa 10 estimacija (stabala) i obučen na trening skupu koristeći funkciju `fit`. Nakon obuke, model je primijenjen na validacijski skup, a predviđanja su spremljena u varijablu `Y_pred`. Izračunata je srednja apsolutna postotna greška između stvarnih i predviđenih vrijednosti, a ona za ovaj model iznosi 0.19127455705199162.
- Random Forest regresija je odabrana jer je pogodna za obradu podataka s velikim brojem karakteristika i mogućnostima za grupiranje i otkrivanje interakcija među varijablama. Vidimo da on postiže bolje rezultate (srednja apsolutna postotna greška je manja). Random Forest model ima veću sposobnost modeliranja nelinearnih veza i može pružiti bolje rezultate kada postoji nelinearna zavisnost između ulaznik i ciljnih varijabli.

3. **Treći model - Linearna Regresija**: Definisana je Linearna regresija `model_LR` i obučena na skupu za treniranje koristeći funkciju `fit`. Nakon obuke, model je primijenjen na validacijski skup, a predikcije su spremljene u varijablu `Y_pred`. Izračunata je srednja apsolutna postotna greška između stvarnih i predviđenih vrijednosti i ona za ovaj model iznosi 0.24448753889588307.
- Linearna regresija je odabrana kao jednostavan i interpretabilan model koji može dati dobar uvid u linearnu vezu između ulaznih i ciljnih varijabli.Ona može biti manje fleksibilna za modeliranje nelinearnih veza, pa su rezultati nešto manje precizni u odnosu na Random Forest model.



## Kako bi se potencijalno poboljšao performans

1. Umjesto uklanjanja varijabli sa NaN vrijednostima, moguće je više pristupa:
    - Umjesto NaN vrijednosti, dodijeliti mean vrijednost date kolone (u ovom slučaju za varijable 'stanje' i 'godina_izgradnje' nije značajno utjecalo na performans, ali je greška bila čak nešto veća)
    - Umjesto NaN vrijednosti dodati novu vrijednost za taj atribut
    - Umjesto dodjele srednje vrijednosti, moguće je koristiti tehnike nadopunjavanja za procjenu nedostajućih vrijednosti na temelju dostupnih podataka. Postoji nekoliko metoda nadopunjavanja, kao što su nadopunjavanje pomoću k-najbližih susjeda ili nadopunjavanje temeljeno na regresiji. 

2. Za nedostajuće adrese ručno dodijetiti približne adrese za one instance za koje se takav podatak nalazi unutar atributa 'opis', kako bi se povećao broj instanci dataseta (ovo bi moglo funkcionirati jer i za one instance za koje je dodijeljena adresa, često nije dodijeljena precizna adresa). 
Da bismo dobili trenutni dataset koji je korišten za obučavanje modela, pored dosad predstavljenog čišćenja podataka, za nekih 500 instanci manuelno je modifikovana vrijednost atributa 'adresa' zbog grešaka u spelovanju i sličnih, kako bi API koji smo koristili ispravno mogao dodijeliti koordinate, a time i odrediti udaljenost od centra.

3. Modifikovanje parametara modela kao što je npr. 'n_estimators' parametar u drugom modelu (nije značajno uticalo na veličinu greške).

4. Pokušati trenirati i druge modele kao što su npr. neuralne mreže.


In [192]:
# First model
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_percentage_error
 
model_SVR = svm.SVR()
model_SVR.fit(X_train,Y_train)
Y_pred = model_SVR.predict(X_valid)
 
print(mean_absolute_percentage_error(Y_valid, Y_pred))

0.3859895797701615


In [195]:
# Second model
from sklearn.ensemble import RandomForestRegressor
 
model_RFR = RandomForestRegressor(n_estimators=150)
model_RFR.fit(X_train, Y_train)
Y_pred = model_RFR.predict(X_valid)
 
mean_absolute_percentage_error(Y_valid, Y_pred)

0.19127455705199162

In [194]:
# Third model
from sklearn.linear_model import LinearRegression
 
model_LR = LinearRegression()
model_LR.fit(X_train, Y_train)
Y_pred = model_LR.predict(X_valid)
 
print(mean_absolute_percentage_error(Y_valid, Y_pred))

0.24448753889588307
